In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
test2 = pd.read_csv("../input/test.csv")
submit = pd.read_csv("../input/Sample_Submission_Tm9Lura.csv")
df.sample(5)

df.dtypes
df.describe()
df.isnull().any()
df.shape

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
150216,1005185,P0095742,F,36-45,4,C,1,0,4,5.0,NaN,2782
160027,1000757,P00158242,M,26-35,12,A,1,0,6,8.0,NaN,16332
389628,1005961,P00295942,F,46-50,0,A,3,0,2,5.0,15.0,13036
409340,1003029,P00173942,M,18-25,4,A,4+,0,1,5.0,6.0,11543
246864,1001998,P00288342,F,51-55,9,B,2,1,8,NaN,NaN,8035


User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,376430.000000,166821.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,9.842329,12.668243,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,5.086590,4.125338,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,2.000000,3.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


User_ID                       False
Product_ID                    False
Gender                        False
Age                           False
Occupation                    False
City_Category                 False
Stay_In_Current_City_Years    False
Marital_Status                False
Product_Category_1            False
Product_Category_2             True
Product_Category_3             True
Purchase                      False
dtype: bool

(550068, 12)

In [2]:
df['Age'].unique()
df['Stay_In_Current_City_Years'].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

array(['2', '4+', '3', '1', '0'], dtype=object)

Catagory to Numerical

In [3]:
df['Gender'] = df['Gender'].map({'M': 0, 'F': 1})
df['Age'] = df['Age'].map({'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6})
df.loc[df['Stay_In_Current_City_Years'] == '4+', 'Stay_In_Current_City_Years'] = '4'
df['City_Category'] = df['City_Category'].map({'A': 0, 'B': 1, 'C': 2})

In [4]:
test['Gender'] = test['Gender'].map({'M': 0, 'F': 1})
test['Age'] = test['Age'].map({'0-17': 0, '18-25': 1, '26-35': 2, '36-45': 3, '46-50': 4, '51-55': 5, '55+': 6})
test.loc[test['Stay_In_Current_City_Years'] == '4+', 'Stay_In_Current_City_Years'] = '4'
test['City_Category'] = test['City_Category'].map({'A': 0, 'B': 1, 'C': 2})

In [5]:
features = ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years',
            'Marital_Status', 'Product_Category_1']
target = ['Purchase']
x_train = df[features]
y_train = df[target]
x_test = test[features]
x_train.head()
y_train.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
0,1,0,10,0,2,0,3
1,1,0,10,0,2,0,1
2,1,0,10,0,2,0,12
3,1,0,10,0,2,0,12
4,0,6,16,2,4,0,8


,Purchase
0,8370
1,15200
2,1422
3,1057
4,7969


In [6]:
from sklearn import linear_model# compute classification accuracy for the linear regression model
from sklearn import metrics # for the check the error and accuracy of the model

lin = linear_model.LinearRegression()
# train the model on the training set
lin.fit(x_train, y_train)
lin_score_train = lin.score(x_train, y_train)
test2['Purchase'] = lin.predict(x_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
test2.to_csv("../output/final_solution.csv", index=False)